Allloy Catalysis Automated Toolkit (ACAT) is a Python package for atomistic modelling of metal (alloy) (oxide) catalysts used in heterogeneous catalysis. The package is based on automatic identifications of adsorption sites and adsorbate coverages on surface slabs and nanoparticles. Synergizing with ASE, ACAT provides useful tools to build atomistic models and perform global optimization tasks for alloy surfaces and nanoparticles with and without adsorbates. The goal is to automate workflows for the high-throughput screening of hetereogeneous catalysts.

ACAT has been developed by Shuang Han at the Section of Atomic Scale Materials Modelling, Department of Energy Conversion and Storage, Technical University of Denmark (DTU). The development is hosted on https://gitlab.com/asm-dtu/acat.

The code can be easily installed using `pip install`:

In [ ]:
!pip install acat asap3

# Adsorption site identification

To begin with, let's generate a ($4\times 4\times 4$) ordered fcc PdAg$_3$(111) surface slab using ASE and visualize it:

In [ ]:
from ase.build import fcc111
from ase.visualize import view

slab_111 = fcc111('Ag', (4,4,4), vacuum=5.)
# Change 1/4 Ag atoms to Pd
for i in [5,7,13,15,16,18,24,26,36,38,44,46,53,55,61,63]:
  slab_111[i].symbol = 'Pd'
# Visualize
view(slab_111, viewer='x3d')

In ASE, there is an ``add_adsorbate`` function that allows adding adsorbate to a surface.

For example, let's add an O* adsorbate to an 'ontop' site with a height of 2 Å and visualize it:

In [ ]:
from ase.build import add_adsorbate
from ase.visualize import view

# Make a copy of the surface slab
atoms = slab_111.copy()
add_adsorbate(atoms, adsorbate='O', height=2., position='ontop')
atoms.center(vacuum=5., axis=2)
# Visualize
view(atoms, viewer='x3d')

This is only made possible because of the relevant information stored in `slab.info['adsorbate_info']` when generating the surface slab using `ase.build.fcc111`. For an `ase.Atoms` object generated elsewhere, ASE cannot identify the adsorption sites, as illustrated below:

In [ ]:
from ase.build import add_adsorbate

# Show the stored relevant information
print(slab_111.info['adsorbate_info'])
atoms = slab_111.copy()
# Delete the information
del atoms.info['adsorbate_info']
# Adding adsorbate now throws an error
try:
  add_adsorbate(atoms, adsorbate='O', height=1.5, position='ontop')
except TypeError as err:
  print(err)

To summarize, ASE has the following limitations for adsorption site identification:
1. surface structure must be generated by `ase.build`;
2. only one pre-defined position for each type of site;
3. does not differentiate two sites of the same type (e.g. ontop) but different elemental compositions (e.g. Ag and Pd);
4. limited number of supported surface types (e.g. metal oxides and many stepped surfaces are not supported);
5. nanoparticles/clusters are not supported.

Other similar codes, e.g. pymatgen and catkit, also require a pre-defined bulk/surface structure to identify adsorption sites, and do not support metal oxides/nanoparticles/clusters.

ACAT overcomes all aforementioned limitations, providing a general way of identifying adsorption sites on an arbitrary surface structure (i.e. any `ase.Atoms` object) that can represent various surfaces and nanoparticles. In detail, ACAT directly implements adsorption site identification for 20 common low-index surfaces (including fcc, bcc and hcp crystal structures) in `acat.adsorption_sites.SlabAdsorptionSites` and for 3 common nanoparticle motifs (fcc, icosahedron, and decahedron) in `acat.adsorption_sites.ClusterAdsorptionSites`. Besides, a general `acat.settings.CustomSurface` class can be used to represent **any** surface strucuture, including (mixed) metal oxides.

What's happening under the hood:

In [ ]:
from google.colab import drive
from IPython.display import Image

drive.mount('/content/drive')
Image('drive/MyDrive/Colab Notebooks/images/acat_algorithms.png')

Using the same PdAg$_3$(111) surface structure as an example, ACAT can now identify a list of information for all adsorption sites:

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites

sas_111 = SlabAdsorptionSites(slab_111, surface='fcc111', composition_effect=True)
sites_111 = sas_111.get_sites()
# Print out the information of each site
for i, site in enumerate(sites_111):
  print('Site {0}: {1}'.format(i, site))

One can use the following script to visualize the positions of all adsorption sites identified by ACAT:

In [ ]:
# Use a larger height for better visualization
def make_dummy(slab, sites, height=0.):
  from ase import Atoms
  # Make a copy of the surface slab
  dummy_atoms = slab.copy()
  # Add a dummy atom to each site
  for site in sites:
    dummy = Atoms('X', positions=[[0,0,0]])
    pos = site['position'] + site['normal'] * height
    dummy.translate(pos - dummy[0].position)
    dummy_atoms += dummy
  return dummy_atoms

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_111 = make_dummy(slab_111, sites_111, height=0.5)
view(dummy_111, viewer='x3d') # ontop sites are probably blocked by larger atoms when visualizing

The information of each adsorption site is stored in a dictionary, where the `'site'`, `'surface'`, `'morphology'`, `'position'`, `'normal'`, `'indices'` and `'composition'` keys contain information of the type, the locating surface, the local geometric environment, the Cartesian coordinate, the normal vector, the atomic indices and the elemental composition of the site, respectively.

For nanoparticles, the adsorption sites can be located at vertices, edges and different types of surfaces.

In [ ]:
from google.colab import drive
from IPython.display import Image

drive.mount('/content/drive')
Image('drive/MyDrive/Colab Notebooks/images/color_facets.png')

ACAT also takes into consideration the geometric environment of adsorption sites on nanoparticles. As an example, let's try to identify the adsorption sites on a truncataed octahedral Pd-Ag nanoalloy:

In [ ]:
from acat.adsorption_sites import ClusterAdsorptionSites
from ase.cluster import Octahedron

# Generate a TOh Pd-Ag nanoalloy
toh = Octahedron('Ag', length=7, cutoff=2)

for atom in toh:
  # Doping
  if atom.index % 2 == 0:
    atom.symbol = 'Pd'
toh.center()
# Identify all adsorption sites on the nanoalloy
cas_toh = ClusterAdsorptionSites(toh, composition_effect=True)
# Print information of each site
sites_toh = cas_toh.get_sites()
for i, site in enumerate(sites_toh):
  print('Site {0}: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_toh = make_dummy(toh, sites_toh, height=0.5)
view(dummy_toh, viewer='x3d')

Sometimes we probably only want to get the adsosrpiton sites on one facet. We can then use the `acat.adsorption_sites.group_sites_by_facet` function to group a given list of sites into different sublists based on their facets, and choose one of the groups.

For example, we can easily classify the adsorption sites on an icosahedral nanoparticle into 20 groups that corrsponds to 20 fcc111 facets:

In [ ]:
from acat.adsorption_sites import ClusterAdsorptionSites, group_sites_by_facet
from ase.cluster import Icosahedron

ih = Icosahedron('Ag', noshells=5)
cas = ClusterAdsorptionSites(ih)
sites_ih = cas.get_sites()
site_groups_ih = group_sites_by_facet(ih, sites_ih)
print('Number of identified facets: {}'.format(len(site_groups_ih)))
for i, site in enumerate(site_groups_ih[0]):
  print('Site {0} on one facet: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_ih = make_dummy(ih, site_groups_ih[0], height=0.5)
view(dummy_ih, viewer='x3d')

For stepped surfaces, the 'morphology' keyword may correspond to a string of either `'step'`, `'terrace'`, `'corner'` or an `'a-b(-c)'` form that represents the intermediate region connecting two of the former three geometric environments. For example, `'sc-tc-h'` represents the region connecting the step chain (sc) and the terrace chain (tc) on a hexagonal (h) lattice. Other Bravais lattice such as orthorhombic (o) and tetragonal (t) can also be identified by ACAT.

As an example, let's try to identify the adsorption sites on a stepped Pd-Ag(211) alloy surface:

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites
from ase.build import fcc211

# Initialize a pure Ag(211) surface slab
slab_211 = fcc211('Ag', (6,3,4), vacuum=5.)
# Generate a Pd-Ag(211) surface slab by changing some Cu to Au
for atom in slab_211:
  if atom.index % 2 == 0:
    atom.symbol = 'Pd'
# Identify all adsorption sites on the alloy slab
sas_211 = SlabAdsorptionSites(slab_211, surface='fcc211', composition_effect=True)
# Print information of each site
sites_211 = sas_211.get_sites()
for i, site in enumerate(sites_211):
  print('Site {0}: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_211 = make_dummy(slab_211, sites_211, height=0.5)
view(dummy_211, viewer='x3d')

Now let's try a stepped bcc(310) Mo-Fe surface that is not supported in ASE:

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites
from ase.io import read

# Download the surface structure file
!wget https://gitlab.com/asm-dtu/acat/-/raw/master/tests/MoFe_310_surface.traj
# Read the surface structure
slab_310 = read('MoFe_310_surface.traj')

# Identify all adsorption sites on the alloy slab
sas_310 = SlabAdsorptionSites(slab_310, surface='bcc310', composition_effect=True)
# Print information of each site
sites_310 = sas_310.get_sites()
for i, site in enumerate(sites_310):
  print('Site {0}: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_310 = make_dummy(slab_310, sites_310, height=0.5)
view(dummy_310, viewer='x3d')

Sometimes it could be useful to identify the adsorption sites on both the top and the bottom terminations of the surface. This can be simply done by setting `both_sides = True`:

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites

# Identify all adsorption sites on the alloy slab
sas_310 = SlabAdsorptionSites(slab_310, surface='bcc310',
                composition_effect=True, both_sides=True)
# Print information of each site
sites_310 = sas_310.get_sites()
for i, site in enumerate(sites_310):
  print('Site {0}: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_310 = make_dummy(slab_310, sites_310, height=0.5)
view(dummy_310, viewer='x3d')

Sometimes it could be useful to identify a set of unique (i.e. symmetry-inequivalent) sites on a surface. This can be especially complicated when the surface has multiple metal components, for example for an alloy A-B(111) surface, the elemental composition of a 3-fold hollow site can be either AAA, AAB, ABB or BBB. In ACAT, this can be simply done by `get_unique_sites()` (remember to set `unique_composition = True` if the elemental composition of each site also needs to be unique):

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites

# Identify all adsorption sites on the alloy slab
sas_310 = SlabAdsorptionSites(slab_310, surface='bcc310', composition_effect=True)
# Print information of each site
unq_sites_310 = sas_310.get_unique_sites(unique_composition=True)
for i, unq_site in enumerate(unq_sites_310):
  print('Unique site {0}: {1}'.format(i, unq_site))

In [ ]:
from ase.visualize import view

# Visualize the sites
unq_dummy_310 = make_dummy(slab_310, unq_sites_310, height=0.5)
view(unq_dummy_310, viewer='x3d')

Same goes for nanoparticles:

In [ ]:
from acat.adsorption_sites import ClusterAdsorptionSites

# Identify all adsorption sites on the nanoalloy
cas_toh = ClusterAdsorptionSites(toh, composition_effect=True)
# Print information of each site
unq_sites_toh = cas_toh.get_unique_sites(unique_composition=True)
for i, unq_site in enumerate(unq_sites_toh):
  print('Unique site {0}: {1}'.format(i, unq_site))

In [ ]:
from ase.visualize import view

# Visualize the sites
unq_dummy_toh = make_dummy(toh, unq_sites_toh, height=0.5)
view(unq_dummy_toh, viewer='x3d')

In case one wants to identify adsorption sites on a surface that is neither an fcc/bcc/hcp structure, the `acat.settings.CustomSurface` class can be used to represent the surface.

For example, we can use a 4-layer anatase TiO$_2$(101) surface structure as reference to represent any 4-layer surface that has the same anatase crystal structure. We can initialize a customized anatase TiO$_2$(101) surface as follows:

In [ ]:
from acat.adsorption_sites import SlabAdsorptionSites
from acat.settings import CustomSurface
from ase.spacegroup import crystal
from ase.build import surface

# Build bulk anatase TiO2
a, c = 3.862, 9.551
anatase = crystal(['Ti', 'O'], basis=[(0 ,0, 0), (0, 0, 0.208)],
          spacegroup=141, cellpar=[a, a, c, 90, 90, 90])
# Cut the anatase TiO2(101) surface
slab_anatase_101 = surface(anatase, (1, 0, 1), 4, vacuum=5)
slab_anatase_101 *= (2,3,1)
# Initialize a CustomSurface object
anatase_101 = CustomSurface(slab_anatase_101, n_layers=4)

Say we have a surface structure resulting from some perturbation (e.g. geometry relaxation) of the reference TiO$_2$(101) surface structure:

In [ ]:
ptb_slab_anatase_101 = slab_anatase_101.copy()
ptb_slab_anatase_101.rattle(stdev=0.2)

Now we can obtain the adsorption sites on this surface structure by passing the `CustomSurface` to `SlabAdsorptionSites`:

In [ ]:
# Identify all adsorption sites on the perturbed anatase TiO2(101) slab
sas_anatase_101 = SlabAdsorptionSites(ptb_slab_anatase_101, surface=anatase_101,
                          composition_effect=True,
                          label_sites=True,
                          both_sides=True)
# Print all sites
sites_anatase_101 = sas_anatase_101.get_sites()
for i, site in enumerate(sites_anatase_101):
  print('Site {0}: {1}'.format(i, site))

In [ ]:
from ase.visualize import view

# Visualize the sites
dummy_anatase_101 = make_dummy(ptb_slab_anatase_101, sites_anatase_101, height=0.5)
view(dummy_anatase_101, viewer='x3d')

# Adsorbate operations

It's time to talk about how to add adsorbates. Once we have a list of all adsorption sites on a surface structure, we can add any adsorbate species to any adsorption site using `acat.build.add_adsorbate_to_site`.

For example, let's add a CH$_3$OH* adsorbate to the PdAg$_3$(111) surface so that the C atom binds to the 2nd site in the list, and visualize it:

In [ ]:
from acat.build import add_adsorbate_to_site
from ase.visualize import view

# Make a copy of the surface slab
atoms_111 = slab_111.copy()
site = sites_111[1]
add_adsorbate_to_site(atoms_111, adsorbate='CH3OH', site=site, height=2.)
view(atoms_111, viewer='x3d')

Since we have a list of all adsorption sites, we can easily add multiple adsorbates to the surface.

For example, we can add O* adsorbates to all fcc sites on the Pd-Ag(211) surface simply using a for loop:

In [ ]:
from acat.build import add_adsorbate_to_site
from ase.visualize import view

# Make a copy of the surface slab
atoms_211 = slab_211.copy()
for site in sites_211:
  if site['site'] == 'fcc':
    add_adsorbate_to_site(atoms_211, adsorbate='O', site=site, height=1.5)
# Visualize
view(atoms_211, viewer='x3d')

We can also generate symmetric surface slabs with adsorbates by setting `both_sides=True`. Below is a more advanced example for generating a symmetric 5-layer rutile TiO$_2$(110) surface slab with N adsorbed at all cus sites:

In [ ]:
from acat.adsorbate_coverage import SlabAdsorbateCoverage
from acat.build import add_adsorbate_to_site
from acat.settings import CustomSurface
from ase.spacegroup import crystal
from ase.build import surface
from ase.visualize import view
from collections import Counter

# Build bulk rutile TiO2
a, c = 4.584, 2.953
rutile = crystal(['Ti', 'O'], basis=[(0, 0, 0), (0.3, 0.3, 0)],
                 spacegroup=136, cellpar=[a, a, c, 90, 90, 90])
# Cut the rutile TiO2(110) surface
slab_rutile_110 = surface(rutile, (1,1,0), layers=5)
# Remove the oxygens above the top surface
indices = [a.index for a in slab_rutile_110 if a.position[2] < 14.]
atoms_rutile_110 = slab_rutile_110[indices]
# Center the slab and expand
atoms_rutile_110.center(vacuum=5., axis=2)
atoms_rutile_110 *= (2,3,1)
# Initialize a CustomSurface object
rutile_110 = CustomSurface(atoms_rutile_110, n_layers=5)
# Identify adsorption sites on both sides with "adsorbate" information
sac_rutile_110 = SlabAdsorbateCoverage(atoms_rutile_110, surface=rutile_110, both_sides=True)
# Get all ontop sites which half are cus sites
ontop_sites = sac_rutile_110.get_sites(site='ontop')
# Get all 3fold sites occupied by oxygen
occupied_3fold_sites = sac_rutile_110.get_sites(occupied=True, site='3fold')
# Cus sites are then the atoms that contribute 4 times to these sites
occurences = [i for st in occupied_3fold_sites for i in st['indices']]
cus_indices = [(i,) for i, count in Counter(occurences).items() if count == 4]
# Add N to all cus sites
for site in ontop_sites:
    if site['indices'] in cus_indices:
        add_adsorbate_to_site(atoms_rutile_110, adsorbate='N', site=site)
# Visualize
view(atoms_rutile_110, viewer='x3d')

Similarly, we can add adsorbates to nanoparticles however we want.

For example, let's add CO* adosrbates to all fcc and 4fold sites on the truncated ocatahedral Pd-Ag nanoalloy:

In [ ]:
from acat.build import add_adsorbate_to_site
from ase.visualize import view

# Make a copy of the nanoalloy
atoms_toh = toh.copy()
for site in sites_toh:
  if site['site'] in ['fcc', '4fold']:
    add_adsorbate_to_site(atoms_toh, adsorbate='CO', site=site, height=1.6)
# Visualize
view(atoms_toh, viewer='x3d')

A more general `add_adsorbate` function is also implemented in ACAT, which allows the addition of an adsorbate to a site featured by its type, surface, morphology, composition or atomic indices.

For example, let's add a CO* adsorbate to an all-Pd step bridge site on the Pd-Ag(211) alloy surface with a height of 1.5 Å:

In [ ]:
from acat.build import add_adsorbate
from ase.visualize import view

# Make a copy of the surface slab
atoms_211 = slab_211.copy()
add_adsorbate(atoms_211, adsorbate='CO', site='bridge', surface='fcc211',
                         composition='PdPd', height=1.5)
# Visualize
view(atoms_211, viewer='x3d')

Alternatively, we can directly specify adding the CO* adsorbate to the site with the atomic indices of `(0,2)`:

In [ ]:
from acat.build import add_adsorbate
from ase.visualize import view

# Make a copy of the surface slab
atoms_211 = slab_211.copy()
add_adsorbate(atoms_211, adsorbate='CO', surface='fcc211',
                         indices=(0,2), height=1.5)
# Visualize
view(atoms_211, viewer='x3d')

Another example: adding a N atom to a Ti-Ti bridge site on an anatase TiO$_2$(101) surface. In this case, ACAT automatically identifies the sites occupied by surface oxygens and then adds the adsorbate to an unoccupied bridge site:

In [ ]:
from acat.build import add_adsorbate
from ase.visualize import view

# Suppose we have a surface structure resulting from some
# perturbation of the reference anatase TiO2(101) surface
atoms_anatase_101 = ptb_slab_anatase_101.copy()
add_adsorbate(atoms_anatase_101, adsorbate='N', site='bridge',
              surface=anatase_101, composition='TiTi')
view(atoms_anatase_101, viewer='x3d')

We can also perform removal of adsorbates from certain adsorption sites using `acat.build.remove_adsorbate_from_site` or `acat.build.remove_adsorbates_from_sites`. Below is an example of removing O species from all cus sites on a rutile TiO2(110) surface slab. Let's first visualize the structure before the removal:

In [ ]:
from ase.visualize import view

# Copy the rutile TiO2(110) slab
atoms_rutile_110 = slab_rutile_110.copy()
atoms_rutile_110.center(vacuum=5., axis=2)
atoms_rutile_110 *= (2,3,1)
# Visualize
view(atoms_rutile_110, viewer='x3d')

Now let's visualize the structure after removal:

In [ ]:
from acat.adsorbate_coverage import SlabAdsorbateCoverage
from acat.build import remove_adsorbates_from_sites
from acat.settings import CustomSurface
from ase.visualize import view
from collections import Counter

# Initialize a CustomSurface object
rutile_110 = CustomSurface(atoms_rutile_110, n_layers=5)
# Identify adsorption sites with "adsorbate" information
sac_rutile_110 = SlabAdsorbateCoverage(atoms_rutile_110, surface=rutile_110)
# Get all ontop sites which half are cus sites
ontop_sites = sac_rutile_110.get_sites(site='ontop')
# Get all 3fold sites occupied by oxygen
occupied_3fold_sites = sac_rutile_110.get_sites(occupied=True, site='3fold')
# Cus sites are then the atoms that contribute 4 times to these sites
occurences = [i for st in occupied_3fold_sites for i in st['indices']]
cus_indices = [(i,) for i, count in Counter(occurences).items() if count == 4]
# Remove O from all cus sites
rm_sites = [s for s in ontop_sites if s['indices'] in cus_indices]
remove_adsorbates_from_sites(atoms_rutile_110, sites=rm_sites)
# Visualize
view(atoms_rutile_110, viewer='x3d')

# Surface adsorbate coverage analysis


Let us consider the situation where we are given a surface structure covered by adsorbates (or is a metal oxide which already has oxygen "adsorbates" on the surface) and we want to analyze which adsorption site is occupied by which adsorbate. In ACAT this kind of surface analysis can be done by `SlabAdsorbateCoverage` for surface slabs and `ClusterAdsorbateCoverage` for nanoparticles.

For example, let's analyze the adsorbate coverage on the Pd-Ag(211) surface covered by O* at all fcc sites (which we have generated previously):

In [ ]:
from acat.adsorbate_coverage import SlabAdsorbateCoverage

# Get information about the surface adsorbate coverage
sac_211 = SlabAdsorbateCoverage(atoms_211, adsorption_sites=sas_211)
# Print the updated adsorpiton sites
hetero_sites_211 = sac_211.get_sites()
for i, site in enumerate(hetero_sites_211):
  print('Site {0}: {1}'.format(i, site))

Apart from the original keys, each updated dictionary now has 8 additional keys $-$ `'bonding_index'`, `'bond_length'`, `'adsorbate'`, `'fragment'`, `'adsorbate_indices'`, `'occupied'`, `'dentate'` and `'fragment_indices'`, which stores information of the index of the atom that bonds with the adsorbate, the adsorbate-metal distance, the name of the adsorbate entity, the name of the adsorbate fragment assigned to the site, the atomic indices of the adsorbate, whether the site is occupied or not, the dentate number and the atomic indices of the adsorbate fragment, respectively.

Using the CH$_3$OH* covered PdAg$_3$(111) surface as an example, if we enumerate all occupied adsorption sites, we can see that the bidentate adsorbate is divided into two fragments - a CH$_3$* fragment that is assigned to a bridge site and an OH* fragment that is assigned to an ontop site:

In [ ]:
from acat.adsorbate_coverage import SlabAdsorbateCoverage

# Get information about the surface adsorbate coverage
sac_111 = SlabAdsorbateCoverage(atoms_111, adsorption_sites=sas_111)
# Print the occupied adsorpiton sites
hetero_sites_111 = sac_111.get_sites(occupied=True)
for i, site in enumerate(hetero_sites_111):
  print('Occupied site {0}: {1}'.format(i, site))

The following example shows the use of `SlabAdsorbateCoverage` to get all unoccupied symmetry-inequivalent adsorption sites on an anatase TiO$_2$(101) surface:

In [ ]:
from acat.adsorbate_coverage import SlabAdsorbateCoverage
from acat.settings import CustomSurface
from ase.spacegroup import crystal
from ase.build import surface

# Build bulk anatase TiO2
a, c = 3.862, 9.551
anatase = crystal(['Ti', 'O'], basis=[(0 ,0, 0), (0, 0, 0.208)],
              spacegroup=141, cellpar=[a, a, c, 90, 90, 90])
# Cut the anatase TiO2(101) surface
slab_anatase_101 = surface(anatase, (1, 0, 1), 4, vacuum=5)
slab_anatase_101 *= (2,3,1)
# Initialize a CustomSurface object
anatase_101 = CustomSurface(slab_anatase_101, n_layers=4)
# Suppose we have a surface structure resulting from some
# perturbation of the reference anatase TiO2(101) surface
ptb_slab_anatase_101 = slab_anatase_101.copy()
ptb_slab_anatase_101.rattle(stdev=0.2)
# Identify all adsorption sites on the perturbed TiO2(101) surface slab
sac_anatase_101 = SlabAdsorbateCoverage(ptb_slab_anatase_101, surface=anatase_101)
# Get all unique unoccupied sites
unq_unoccupied_sites = sac_anatase_101.get_unique_sites(occupied=False)
for i, site in enumerate(unq_unoccupied_sites):
  print('Unique unoccupied site {0}: {1}'.format(i, site))

For nanoparticles, we can use `acat.adsorbate_coverage.ClusterAdsorbateCoverage` instead:

In [ ]:
from acat.adsorbate_coverage import ClusterAdsorbateCoverage

# Get information about the surface adsorbate coverage
cac_toh = ClusterAdsorbateCoverage(atoms_toh, adsorption_sites=cas_toh)
# Print the occupied adsorpiton sites
hetero_sites_toh = cac_toh.get_sites(occupied=True)
for i, site in enumerate(hetero_sites_toh):
  print('Occupied site {0}: {1}'.format(i, site))

# Structure generators

To generate alloy surfaces or nanoparticles with random chemical orderings, one can use `acat.build.ordering.RandomOrderingGenerator`.

For example, let's generate 20 Pd-Ag(211) surfaces with random compositions and chemical orderings:

In [ ]:
from acat.build.ordering import RandomOrderingGenerator as ROG
from ase.build import fcc211
from ase.io import read
from ase.visualize import view

# Initialize a pure Ag(211) surface slab prototype
proto = fcc211('Ag', (6,3,4), vacuum=5.)
# Generate random chemical orderings
rog = ROG(proto, elements=['Pd','Ag'],
                   trajectory='orderings_211.traj')
rog.run(num_gen=20)
# Visualize the output structures
alloys_211 = read('orderings_211.traj', index=':')
view(alloys_211[-1], viewer='x3d')  # unfortunately we can only visualize one structure at a time on google colab

We can also specify a certain alloy composition in ACAT. For example, let's generate 20 icosahedral equiatomic Ir$_{0.2}$Pd$_{0.2}$Pt$_{0.2}$Rh$_{0.2}$Ru$_{0.2}$ high-entropy nanoalloys with random chemical orderings:

In [ ]:
from acat.build.ordering import RandomOrderingGenerator as ROG
from ase.cluster import Icosahedron
from ase.io import read
from ase.visualize import view

# Initialize a pure Ih nanoparticle prototype
proto = Icosahedron('Ir', noshells=4)
# Generate random chemical orderings
rog = ROG(proto, elements=['Ir','Pd','Pt','Rh','Ru'],
                   composition={'Ir':0.2,'Pd':0.2,'Pt':0.2,'Rh':0.2,'Ru':0.2},
                   trajectory='orderings_ih.traj')
rog.run(num_gen=20)
# Visualize the output structures
alloys_ih = read('orderings_ih.traj', index=':')
view(alloys_ih[-1], viewer='x3d')

ACAT can also be used to generate random mixed metal oxides with some simple work around. Below is an example of generating 20 random equiatomic anatase Ti$_{0.2}$Ni$_{0.2}$Pd$_{0.2}$Cu$_{0.2}$Au$_{0.2}$O$_{2}$(101) high-entropy oxide surfaces:

In [ ]:
from acat.build.ordering import RandomOrderingGenerator as ROG
from ase.spacegroup import crystal
from ase.build import surface
from ase.io import read
from ase.visualize import view

# Build bulk anatase TiO2
a, c = 3.862, 9.551
anatase = crystal(['Ti', 'O'], basis=[(0 ,0, 0), (0, 0, 0.208)],
               spacegroup=141, cellpar=[a, a, c, 90, 90, 90])
# Cut the anatase TiO2(101) surface
proto = surface(anatase, (1, 0, 1), 4, vacuum=5)
# Expand and center
proto *= (2,3,1)
proto.center(axis=2)
# Distinguish metals and oxygens
metal_ids, oxygen_ids = [], []
for i, atom in enumerate(proto):
    if atom.symbol == 'O':
        oxygen_ids.append(i)
    else:
        metal_ids.append(i)
metal_proto, oxygen_atoms = proto[metal_ids], proto[oxygen_ids]
# Prepare sorted_ids for sorting atomic indices back to original order
sorted_ids = [0] * len(proto)
for k, v in enumerate(metal_ids + oxygen_ids):
    sorted_ids[v] = k
# Generate random metal orderings
rog = ROG(metal_proto, elements=['Ti','Ni','Pd','Cu','Au'],
          composition={'Ti':0.2,'Ni':0.2,'Pd':0.2,'Cu':0.2,'Au':0.2})
rog.run(num_gen=50)
alloys = read('orderings.traj', index=':')
# Sort back to original order
mixed_oxides = [(a + oxygen_atoms)[sorted_ids] for a in alloys]
# Visualize the output structures
view(mixed_oxides[-1], viewer='x3d')

Besides high-entropy alloys/oxides, we can also generate high-symmetry alloys. For high-symmetry nanoalloys, we can specify the the symmetry of the chemical ordering using `acat.build.ordering.SymmetricClusterOrderingGenerator`, such as `'spherical'`, `'cylindrical'`, `'planar'`, `'mirror planar'`, `'circular'` and `'mirror circular'`.

For example, let's generate 20 high-symmetry decahedral Au-Pd-Ag nanoalloys with mirror circular symmetry:

In [ ]:
from acat.build.ordering import SymmetricClusterOrderingGenerator as SCOG
from ase.cluster import Decahedron
from ase.io import read
from ase.visualize import view

# Initialize a pure Ag Dh nanoparticle as the prototype
proto = Decahedron('Ag', p=3, q=3, r=1)
# Randomly generate 20 nanoalloys with mirror circular symmetry
scog = SCOG(proto, elements=['Au','Pd','Ag'],
                       symmetry='mirror_circular',
                       trajectory='orderings_dh.traj')
scog.run(max_gen=20, mode='stochastic')
# Visualize the output structures
alloys_dh = read('orderings_dh.traj', index=':')
view(alloys_dh[-1], viewer='x3d')

Again, ACAT allows the specification of the composition, for example generating a high-symmetry Au$_{0.2}$Pd$_{0.2}$Ag$_{0.6}$ nanoalloy (note that it could be impossible to form any symmetric nanoalloy at certain compositions):

In [ ]:
from acat.build.ordering import SymmetricClusterOrderingGenerator as SCOG
from ase.cluster import Decahedron
from ase.io import read
from ase.visualize import view

# Initialize a pure Ag Dh nanoparticle as the prototype
proto = Decahedron('Ag', p=3, q=3, r=1)
# Randomly generate 20 nanoalloys with cylindrical symmetry
scog = SCOG(proto, elements=['Au','Pd','Ag'],
                       symmetry='cylindrical',
                       trajectory='orderings_dh.traj',
                       composition={'Au':0.2,'Pd':0.2,'Ag':0.6})
scog.run(max_gen=20, mode='stochastic')
# Visualize the output structures
alloys_dh = read('orderings_dh.traj', index=':')
view(alloys_dh[-1], viewer='x3d')

ACAT also provides tools for generating adsorbate coverage patterns.

For example, we can use `acat.build.adlyer.RandomPatternGenerator` to randomly generate 20 adsorbate coverage patterns on a decahedral Ag nanoparticle with 6 adsorbates that can be either H\*, O\* or OH\* with a minimum adsorbate-adsorbate distance of 2 Å:

In [ ]:
from acat.build.adlayer import RandomPatternGenerator as RPG
from acat.adsorption_sites import ClusterAdsorptionSites
from ase.cluster import Decahedron
from ase.io import read
from ase.visualize import view

# Initialize a pure Ag Dh nanoparticle
proto = Decahedron('Ag', p=2, q=2, r=1)
# First identify the adsorption sites to speed up the generation
cas_dh = ClusterAdsorptionSites(proto, composition_effect=False)
# Generate random adsorbate coverage patterns
rpg = RPG(proto, adsorbate_species=['H','O','OH'],
                  min_adsorbate_distance=3.,
                  adsorption_sites=cas_dh,
                  trajectory='patterns_dh.traj')
rpg.run(num_gen=20, action='add', num_act=6)
# Visualize the output structures
images_dh = read('patterns_dh.traj', index=':')
view(images_dh[-1], viewer='x3d')

Or we can try it on the 20 Pd-Ag(211) alloy surfaces that we have already generated, but this time assign a probability of adding each species, for example 20\% H\*, 60\% O\*, 20\% OH\*:

In [ ]:
from acat.build.adlayer import RandomPatternGenerator as RPG
from acat.adsorption_sites import SlabAdsorptionSites
from ase.cluster import Decahedron
from ase.io import read
from ase.visualize import view

# First identify the adsorption sites to speed up the generation
sas_211 = SlabAdsorptionSites(alloys_211[0], surface='fcc211', composition_effect=True)
# Generate random adsorbate coverage patterns
rpg = RPG(alloys_211, adsorbate_species=['H','O','OH'],
                  species_probabilities={'H':0.2,'O':0.6,'OH':0.2},
                  min_adsorbate_distance=2.,
                  adsorption_sites=sas_211,
                  trajectory='patterns_211.traj')
rpg.run(num_gen=20, action='add', num_act=6)
# Visualize the output structures
images_211 = read('patterns_211.traj', index=':')
view(images_211[-1], viewer='x3d')

Instead of generating adsorbate coverage patterns completely randomly, sometimes it could be useful to introduce some heuristics so that the generated patterns are more reasonable. In `acat.build.adlayers`, there are 3 functions implemented for this purpose $-$ `max_dist_coverage_pattern`, `min_dist_coverage_pattern` and `special_coverage_pattern`.

`max_dist_coverage_pattern` uses a clustering algorithm (implemented in `pyclustering`) to generate adsorbate coverage patterns that maximize the minimum adsorbate-adsorbate distance given the positions of adsorption sites and the number of adsorbates to be added.

For example, let's use `max_dist_coverage_pattern` to generate a random CO\* coverage pattern on the PdAg$_3$(111) surface with a coverage of 0.67 ML:

In [ ]:
from acat.build.adlayer import max_dist_coverage_pattern as maxdcp
from ase.visualize import view
try:
  import pyclustering
except ModuleNotFoundError:
  import pip
  pip.main(['install', 'pyclustering'])

# Make a copy of the surface slab
atoms = slab_111.copy()
# Generate a random 0.67 ML CO coverage pattern
pattern = maxdcp(atoms, adsorbate_species='CO', coverage=0.67, surface='fcc111')
view(pattern, viewer='x3d')

On the other hand, `min_dist_coverage_pattern` maximizes the density of the adsorbates when a minimum adsorbate-adsorbate distance constraint is given. Note that the number of adsorbates generated by this function is not fixed.

For example, let's use `min_dist_coverage_pattern` to generate a random CO\* coverage pattern on the PdAg$_3$(111) surface with a minimum adsorbate-adsorbate distance of 3 Å:

In [ ]:
from acat.build.adlayer import min_dist_coverage_pattern as mindcp
from ase.visualize import view

# Make a copy of the surface slab
atoms = slab_111.copy()
# Generate a random CO coverage pattern
pattern = mindcp(atoms, adsorbate_species='CO', min_adsorbate_distance=3., surface='fcc111')
view(pattern, viewer='x3d')

`special_coverage_pattern` is implemented for generating well-defined ordered patterns at the 4 most representative adsorbate coverages: 0.25 ML, 0.5 ML, 0.75 ML and 1 ML, but we plan to implement more ordered patterns in the future.

For example, we can generate the 0.5 ML CO\* coverage pattern ("honeycomb") on the PdAg$_3$(111) surface by

In [ ]:
from acat.build.adlayer import special_coverage_pattern as scp
from ase.visualize import view

# Make a copy of the surface slab
atoms = slab_111.copy()
# Generate the ordered 0.5 ML CO coverage pattern
pattern = scp(atoms, adsorbate_species='CO', coverage=0.5, surface='fcc111')
view(pattern, viewer='x3d')

As another example, we can generate the 0.75 ML CO* coverage pattern on an fcc100 surface by

In [ ]:
from acat.build.adlayer import special_coverage_pattern as scp
from ase.build import fcc100
from ase.visualize import view

# Generate an fcc100 surface slab
atoms = fcc100('Ag', (4,4,4), vacuum=5.)
# Generate the ordered 0.75 ML CO coverage pattern
pattern = scp(atoms, adsorbate_species='CO', coverage=0.75, surface='fcc100')
view(pattern, viewer='x3d')

This function is even applicable to various types of nanoparticles:

In [ ]:
from acat.build.adlayer import special_coverage_pattern as scp
from ase.cluster import Octahedron
from ase.visualize import view

# Generate an octahedral nanoparticle
atoms = Octahedron('Ag', length=11, cutoff=5)
# Generate the ordered 0.75 ML CO coverage pattern
pattern = scp(atoms, adsorbate_species='CO', coverage=0.75)
view(pattern, viewer='x3d')

In [ ]:
from acat.build.adlayer import special_coverage_pattern as scp
from ase.cluster import Decahedron
from ase.visualize import view

# Generate an decahedral nanoparticle
atoms = Decahedron('Ag', p=4, q=3, r=1)
# Generate the ordered 0.5 ML CO coverage pattern
pattern = scp(atoms, adsorbate_species='CO', coverage=0.5,
                      min_adsorbate_distance=2.)
view(pattern, viewer='x3d')

In [ ]:
from acat.build.adlayer import special_coverage_pattern as scp
from ase.cluster import Icosahedron
from ase.visualize import view

# Generate an icosahedral nanoparticle
atoms = Icosahedron('Ag', noshells=5)
# Generate the ordered 0.25 ML CO coverage pattern
pattern = scp(atoms, adsorbate_species='CO', coverage=0.25)
view(pattern, viewer='x3d')

# Atomistic Global optimization

In this session we will use genetic algorithm (GA) to perform global optimization of atomic structures. GA is a metaheuristic method inspired by natural selection and survival of the fittest, which belongs to the larger class of evolutionary algorithms (EAs). GAs are commonly used to locate the global optimum by using biologically inspired operators such as mutation, crossover and selection.

First let's consider the problem of finding the most stable truncated octahedral Pd$_{35}$Ag$_{105}$ nanoalloy configuration. We can start with a walkthrough of running a standard GA using the `ase.ga` module. Note that we will use a semi-empirical EMT potential for fast energy evaluations in the GAs.

In [ ]:
# Start by importing all relevant modules
from acat.build.ordering import RandomOrderingGenerator as ROG
from acat.build.ordering import SymmetricClusterOrderingGenerator as SCOG
from acat.build.adlayer import min_dist_coverage_pattern as mindcp
from acat.adsorption_sites import SlabAdsorptionSites
from acat.adsorbate_coverage import SlabAdsorbateCoverage
from ase.cluster import Icosahedron, Octahedron, Decahedron
from ase.build import fcc111
from ase.geometry import get_layers
from ase.ga.data import DataConnection, PrepareDB
from ase.ga.particle_mutations import RandomPermutation,COM2surfPermutation
from ase.ga.particle_mutations import Rich2poorPermutation, Poor2richPermutation
from ase.ga.particle_crossovers import CutSpliceCrossover
from acat.ga.group_operators import GroupSubstitute, GroupPermutation, GroupCrossover
from acat.ga.adsorbate_operators import AddAdsorbate, RemoveAdsorbate, MoveAdsorbate
from acat.ga.adsorbate_operators import ReplaceAdsorbateSpecies, CatalystAdsorbateCrossover
from acat.ga.slab_operators import RandomSlabPermutation
from ase.ga.offspring_creator import OperationSelector
from ase.ga.particle_comparator import NNMatComparator
from acat.ga.graph_comparators import WLGraphComparator
from ase.ga.population import Population
from acat.ga.multitasking import MultitaskPopulation
from ase.ga.convergence import GenerationRepetitionConvergence
from ase.ga.utilities import closest_distances_generator, get_nnmat
from acat.utilities import neighbor_shell_list, get_adj_matrix
from ase.data import atomic_numbers
from ase.io import read, write
from ase.visualize import view
from random import randint
from ase.calculators.emt import EMT
from ase.calculators.singlepoint import SinglePointCalculator as SPC
from ase.optimize import BFGS
import matplotlib.pyplot as plt
import numpy as np
import shutil
import math
import os

## Initialize a database

First, we need to define the population size of each generation in GA:

In [ ]:
# Define population
pop_size = 50

Now generate the initial population using the `RandomOrderingGenerator` implemented in ACAT:

In [ ]:
# Define the bimetallic species
bimetallic_species = ['Pd', 'Ag']

# Initialize a nanoparticle with a certain shape (Icosahedron / Octahedron / Decahedron)
proto = Octahedron('Ag', length=6, cutoff=1)
proto.center(vacuum=5.)

# Generate 50 icosahedral Pd35Ag105 nanoparticles with random orderings
rog = ROG(proto, elements=bimetallic_species,
                  composition={'Pd':35/140,'Ag':105/140},
                  trajectory='starting_generation_standard.traj')
rog.run(num_gen=pop_size)

Visualize the starting generation:

In [ ]:
images = read('starting_generation_standard.traj', index=':')
view(images[-1], viewer='x3d')

Prepare a database to save structures during the GA run:

In [ ]:
# Instantiate the db
db_name = 'Pd35Ag105_standard.db'

os.system('rm {}'.format(db_name))
db = PrepareDB(db_name, cell=images[0].cell, population_size=pop_size)

for atoms in images:
  db.add_unrelaxed_candidate(atoms)

# Connect to the db
db = DataConnection(db_name)

## Define GA parameters

Define the standard genetic operators. Here the mutations are done by changing the type of a metal atom to another type, and the crossover is done by cutting two nanoalloys into halves then taking one from each parent and splicing them into a new nanoalloy.

In [ ]:
# Define closest distance after crossover
cd = closest_distances_generator(atom_numbers=[atomic_numbers[bimetallic_species[0]],
                                                       atomic_numbers[bimetallic_species[1]]],
                                                       ratio_of_covalent_radii=0.7)

soclist = ([3, 2, 1, 1, 3], # Define the ratios of different operators
       [RandomPermutation(elements=bimetallic_species, num_muts=randint(1, 10)),
       COM2surfPermutation(elements=bimetallic_species, min_ratio=0.1, num_muts=5),
       Rich2poorPermutation(elements=bimetallic_species),
       Poor2richPermutation(elements=bimetallic_species),
       CutSpliceCrossover(cd, keep_composition=True)])
op_selector = OperationSelector(*soclist)

Define comparator to prevent duplicates entering the population:

In [ ]:
comp = NNMatComparator(0.2, bimetallic_species)

Define the population:

In [ ]:
pop = Population(data_connection=db, population_size=pop_size, comparator=comp)

Define the convergence criterion:

In [ ]:
cc = GenerationRepetitionConvergence(pop, 2)

Define the energy minimzation function used in the GA. The minimized energy is used as the score (i.e. fitness) of each candidate.

In [ ]:
def relax(atoms):
    # Centering the atoms
    atoms.center(vacuum=5.)
    # Here we use EMT calcualtor for efficiency
    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.1)
    # Get the minimized EMT energy
    e = atoms.get_potential_energy()
    atoms.info['data']['nnmat'] = get_nnmat(atoms)
    atoms.info['key_value_pairs']['raw_score'] = -e
    atoms.info['key_value_pairs']['EMT_energy'] = e

## Run the GA

Relax the starting generation. See if your energy minimization function is working. The starting generation will take longer time to relax.

In [ ]:
ncand = 1
e_gen1 = 1e5
atoms_gen1 = None
for atoms in db.get_all_unrelaxed_candidates():
    print('\rRelaxing candidate {}'.format(ncand), end='')
    relax(atoms)
    db.add_relaxed_step(atoms)
    e = atoms.info['key_value_pairs']['EMT_energy']
    if e < e_gen1:
        e_gen1 = e
        atoms_gen1 = atoms
    ncand += 1
pop.update()

To track the progress of the GA run, we make 2 lists that store the minimum energies and the corresponding structures for each generation.

In [ ]:
e_gm = [e_gen1]
atoms_gm = [atoms_gen1]

Specify the maximum number of generations:

In [ ]:
max_gens = 200

Run the main GA program:

In [ ]:
# Below is the iterative part of the algorithm
gen_num = db.get_generation_number()
ncand = gen_num * pop_size + 1
for i in range(max_gens):
    # Check if converged
    if cc.converged():
        print('Converged')
        break

    print('\nCreating and evaluating generation {0}'.format(gen_num + i + 1))
    new_generation = []
    e_gen = 1e5
    atoms_gen = None
    for _ in range(pop_size):
        # Select an operator and use it
        op = op_selector.get_operator()
        # Select parents for a new candidate
        p1, p2 = pop.get_two_candidates()
        parents = [p1, p2]
        offspring, desc = op.get_new_individual(parents)
        # An operator could return None if an offspring cannot be formed
        # by the chosen parents
        if offspring is None:
            continue

        print('\rRelaxing candidate {0}'.format(ncand), end='')
        relax(offspring)
        e = offspring.info['key_value_pairs']['EMT_energy']
        # Update the global minimum of the current generation
        if e < e_gen:
          e_gen = e
          atoms_gen = offspring
        new_generation.append(offspring)
        ncand += 1

    # Append to the list of global minima
    e_gm.append(e_gen)
    atoms_gm.append(atoms_gen)
    # Plot the current minima
    x = list(range(gen_num + i + 2 - len(e_gm), gen_num + i + 2))
    y = e_gm
    plt.xticks(range(min(x), math.ceil(max(x)) + 1))
    plt.xlabel('Generation')
    plt.ylabel('Energy / eV')
    plt.plot(x, y, marker='.', markersize=10)
    plt.show()

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    db.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update(new_generation)

Visualize the evolution of the global minimum structure:

In [ ]:
e_gm_standard = e_gm.copy()
atoms_gm_standard = atoms_gm.copy()
view(atoms_gm_standard)

## Imposing symmetry constraints

To accelerate the GA search, we can instead only search for symmetry nanoalloys. The `SymmetricOrderingGenerator` is needed to generate an initial population of symmetric nanoalloys, and a set of symmetry-constrained genetic operators is needed to preserve the symmetry throughout the GA run.

Using the same setting but with symmetry constraints, we can adapt the code to the following:

In [ ]:
# Define population
pop_size = 50
# Define the bimetallic species
bimetallic_species = ['Pd', 'Ag']

# Initialize a nanoparticle with a certain shape (Icosahedron / Octahedron / Decahedron)
proto = Octahedron('Ag', length=6, cutoff=1)
proto.center(vacuum=5.)

##################################################################################
# Randomly generate 50 nanoalloys with mirror circular symmetry
scog = SCOG(proto, elements=bimetallic_species,
                       symmetry='mirror_circular',
                       composition={'Pd':35/140,'Ag':105/140},
                       trajectory='starting_generation_symmetric.traj')
scog.run(max_gen=pop_size, mode='stochastic')
groups = scog.get_groups()
images = read('starting_generation_symmetric.traj', index=':')
##################################################################################

# Instantiate the db
db_name = 'Pd35Ag105_symmetric.db'

os.system('rm {}'.format(db_name))
db = PrepareDB(db_name, cell=images[0].cell, population_size=pop_size)

for atoms in images:
    db.add_unrelaxed_candidate(atoms)

# Connect to the db
db = DataConnection(db_name)

##################################################################################
# Define symmetry-constrained genetic operators
soclist = ([2, 3],
       [GroupPermutation(groups, elements=bimetallic_species, num_muts=1),
        GroupCrossover(groups, elements=bimetallic_species,  keep_composition=True),])
op_selector = OperationSelector(*soclist)
##################################################################################

comp = NNMatComparator(0.2, bimetallic_species)

pop = Population(data_connection=db, population_size=pop_size, comparator=comp)
cc = GenerationRepetitionConvergence(pop, 2)

def relax(atoms):
    # Centering the atoms
    atoms.center(vacuum=5.)
    # Here we use EMT calcualtor for efficiency
    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.1)
    # Get the minimized EMT energy
    e = atoms.get_potential_energy()
    atoms.info['data']['nnmat'] = get_nnmat(atoms)
    atoms.info['key_value_pairs']['raw_score'] = -e
    atoms.info['key_value_pairs']['EMT_energy'] = e

ncand = 1
e_gen1 = 1e5
atoms_gen1 = None
for atoms in db.get_all_unrelaxed_candidates():
    print('\rRelaxing candidate {}'.format(ncand), end='')
    relax(atoms)
    db.add_relaxed_step(atoms)
    e = atoms.info['key_value_pairs']['EMT_energy']
    if e < e_gen1:
      e_gen1 = e
      atoms_gen1 = atoms
    ncand += 1
pop.update()

e_gm = [e_gen1]
atoms_gm = [atoms_gen1]
max_gens = 200

# Below is the iterative part of the algorithm
gen_num = db.get_generation_number()
ncand = gen_num * pop_size + 1
for i in range(max_gens):
    # Check if converged
    if cc.converged():
        print('Converged')
        break

    print('\nCreating and evaluating generation {0}'.format(gen_num + i + 1))
    new_generation = []
    e_gen = 1e5
    atoms_gen = None
    for _ in range(pop_size):
        # Select an operator and use it
        op = op_selector.get_operator()
        # Select parents for a new candidate
        p1, p2 = pop.get_two_candidates()
        parents = [p1, p2]
        offspring, desc = op.get_new_individual(parents)
        # An operator could return None if an offspring cannot be formed
        # by the chosen parents
        if offspring is None:
            continue

        print('\rRelaxing candidate {0}'.format(ncand), end='')
        relax(offspring)
        e = offspring.info['key_value_pairs']['EMT_energy']
        # Update the global minimum of the current generation
        if e < e_gen:
            e_gen = e
            atoms_gen = offspring
        new_generation.append(offspring)
        ncand += 1

    # Append to the list of global minima
    e_gm.append(e_gen)
    atoms_gm.append(atoms_gen)
    # Plot the current minima
    x = list(range(gen_num + i + 2 - len(e_gm), gen_num + i + 2))
    y = e_gm
    plt.xticks(range(min(x), math.ceil(max(x)) + 1))
    plt.xlabel('Generation')
    plt.ylabel('Energy / eV')
    plt.plot(x, y, marker='.', markersize=10)
    plt.show()

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    db.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update(new_generation)

Visualize the evolution of the global minimum structure and compare the final structure with the one obtained from the standard GA.

In [ ]:
e_gm_symmetric = e_gm.copy()
atoms_gm_symmetric = atoms_gm.copy()
view(atoms_gm_symmetric[-1], viewer='x3d')

Plot and compare the convergence curves between the standard GA and the symmetry-constrained GA:

In [ ]:
plt.plot(range(len(e_gm_standard)), e_gm_standard, labe='standard GA')
plt.plot(range(len(e_gm_symmetric)), e_gm_symmetric, labe='symmetry-constrained GA')
plt.xlabel('Generation')
plt.ylabel('Global minimum energy / eV')

## Global optimization under a given reaction condition

Now let's consider the problem of finding the most stable alloy structure under a given reaction condition. Compared to the previous examples, the biggest difference here is that we need to include adsorbates in the global optimization due to the reactive environment.

As an example, we will try to find the most stable PdAg$_3$(111) structure under the electrochemical oxygen reduction reaction (ORR) condition of $U_{\mathrm{RHE}}$ = 0 V ($U_{\mathrm{RHE}}$ is the reversible hydrogen electrode potential). For each PdAg$_3$(111) alloy surface, we fix the bottom layer to be the same as that of a reference stoichiometric PdAg$_3$(111) surface. In this example, we will only consider O* and H* adsorbate species as the EMT potential cannot describe OH* or OOH* well enough. The stability of the system can then be described by the surface free energy
\begin{equation}
\gamma = \frac{1}{A} \left[G - N_{\mathrm{Pd}}\mu_{\mathrm{Pd}} - N_{\mathrm{Ag}}\mu_{\mathrm{Ag}} - N_{\mathrm{H}}\mu_{\mathrm{H}} - N_{\mathrm{O}}\mu_{\mathrm{O}} \right] - \gamma_{\mathrm{ref}}
\end{equation}
where $A$ is the surface area, $G$ is the solid phase Gibbs free energy including the surface and the adsorbates,
$\gamma_{\mathrm{ref}}$ is the surface free energy of the clean stoichiometric PdAg$_3$(111) slab which represents the surface free energy contribution of the bottom layer.

We assume the alloy surface is always in equilibrium with the protons and liquid
water in the aqueous phase. This allows us to write their chemical potentials as
\begin{equation}
\mu_{\mathrm{O}} = \mu_{\mathrm{H_2O(l)}} - 2\left(\mu_{\mathrm{H^+(aq)}} + \mu_{\mathrm{e^-}}\right)
\end{equation}
\begin{equation}
\mu_{\mathrm{H}} = \mu_{\mathrm{H^+(aq)}} + \mu_{\mathrm{e^-}}
\end{equation}
Using the computational hydrogen electrode (CHE), we can calculate the $\mu_{\mathrm{H^+(aq)}} + \mu_{\mathrm{e^-}}$ using the following pH-independent expression:
\begin{equation}
\mu_{\mathrm{H^+(aq)}} + \mu_{\mathrm{e^-}} = \frac{1}{2} \mu_{\mathrm{H_2(g)}} - eU_{\mathrm{RHE}}
\end{equation}
Now we can write the surface free energy as
\begin{equation}
\gamma = \frac{1}{A} \left[G - N_{\mathrm{Pd}}\mu_{\mathrm{Pd}} - N_{\mathrm{Ag}}\mu_{\mathrm{Ag}} - N_{\mathrm{O}}\mu_{\mathrm{H_2O(l)}} - (N_{\mathrm{H}} - 2N_{\mathrm{O}})(\mu_{\mathrm{H_2(g)}} - U_{\mathrm{RHE}}) \right] - \gamma_{\mathrm{ref}}
\end{equation}

For simplicity, we assume the composition of the PdAg$_3$(111) surface is fixed at Pd:Ag = 1:3, but the chemical ordering of the alloy surface and the number of surface O and H atoms are allowed to vary. Evidently, we can write the fitness function of each PdAg$_3$(111) surface structure as
\begin{align}
f = -\left[G - N_{\mathrm{O}}\mu_{\mathrm{H_2O(l)}} - (N_{\mathrm{H}} - 2N_{\mathrm{O}})(\mu_{\mathrm{H_2(g)}} - U_{\mathrm{RHE}})\right]
\end{align}
Here we will approximate it with
\begin{align}
f = -\left[E - N_{\mathrm{O}}\mu_{\mathrm{H_2O(l)}} - (N_{\mathrm{H}} - 2N_{\mathrm{O}})(\mu_{\mathrm{H_2(g)}} - U_{\mathrm{RHE}})\right]
\end{align}
where $E$ is the potential energy of the adsorbates-covered surface slab, $\mu_{\mathrm{H_2O(l)}}$ and $\mu_{\mathrm{H_2(g)}}$ are taken to be 1.647 eV and 1.184 eV, respectively (all calculated by EMT).

To start with, let's specify the population size, the metal and adsorbate species, and the reaction condition：

In [ ]:
# Define population
pop_size = 50
# Define the bimetallic species
bimetallic_species = ['Pd', 'Ag']
# Define the adsorbate species
adsorbate_species = ['O', 'H']
# Define the reaction condition
U = 0

Now let's generate an initial population of clean alloy surface slabs with fixed composition and visualize them:  

In [ ]:
# Initialize a pure fcc111 surface slab
proto = fcc111('Ag', (4,4,4), vacuum=5.)
# Change 1/4 Ag atoms to Pd
for i in [5,7,13,15,16,18,24,26,36,38,44,46,53,55,61,63]:
  proto[i].symbol = 'Pd'
proto_bot, proto_top = proto[:16], proto[16:]

# Generate 50 PdAg3(111) slabs with random orderings
rog = ROG(proto_top, elements=bimetallic_species,
                  composition={'Pd':0.25,'Ag':0.75},
                  trajectory='starting_slabs_single.traj')
rog.run(num_gen=pop_size)

top_slabs = read('starting_slabs_single.traj', index=':')
slabs = [proto_bot+a for a in top_slabs]
view(slabs[-1], viewer='x3d')

We can now get the information of the adsorption sites. Note that since we are only searching fcc111 surface structures and the atomic indices of the slab are always preserved, we can reuse the `SlabAdsorptionSites` object during the GA to accelerat the search. Also, since we have the prior knowledge that the bridge sites are unstable, we can further accelerate the search by ignoring them during the search.

In [ ]:
# Get the adsorption sites. Composition does not affect GA operations
sas = SlabAdsorptionSites(slabs[0], surface='fcc111', ignore_sites='bridge',
                                           composition_effect=False)

We can also get the indices of the atoms in the bottom layer which should be fixed and the top 3 layers which are allowed to mutate:

In [ ]:
# Get the atom indexes at different layers
layers = get_layers(slabs[0], (0, 0, 1), tolerance=0.3)[0] # Start from bottom
bot_indices = np.argwhere(layers == 0).ravel().tolist()
mut_indices = np.argwhere(layers > 0).ravel().tolist()

Now let's complete the generation of our initial population by randomly adding adsorbates to the alloy surface slabs that we have just generated, and visualize them:

In [ ]:
images = []
for slab in slabs:
  # Generate a random O* coverage pattern (set a random
  # min_adsorbate_distance to generate different coverages)
  image = mindcp(slab, adsorbate_species=adsorbate_species, adsorption_sites=sas,
                            min_adsorbate_distance=np.random.uniform(2.2,5.2))
  images.append(image)
write('starting_generation_single.traj', images)
view(images[-1], viewer='x3d')

Instantiate the database and add the initial structures into the database:

In [ ]:
# Instantiate the db
db_name = 'PdAg3_single.db'

os.system('rm {}'.format(db_name))
db = PrepareDB(db_name, cell=images[0].cell, population_size=pop_size)

for atoms in images:
    db.add_unrelaxed_candidate(atoms)

# Connect to the db
db = DataConnection(db_name)

Define the genetic operators. The mutations now not only can be done by mutating metal atoms, but also by adding/removing/moving/replacing an O\*/H\* adsorbate. The crossover is now done by cutting through the interfaces of two structures, taking the adlayer of one structure and the alloy surface slab of another, then splicing them together.

In [ ]:
soclist = ([3, 1, 1, 1, 1, 3],
       [RandomSlabPermutation(allowed_indices=mut_indices),
       AddAdsorbate(adsorbate_species, adsorption_sites=sas,
                              max_coverage=1., min_adsorbate_distance=2.2,
                              heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       RemoveAdsorbate(adsorbate_species, adsorption_sites=sas),
       MoveAdsorbate(adsorbate_species, adsorption_sites=sas,
                                  min_adsorbate_distance=2.2,
                                  heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       ReplaceAdsorbateSpecies(adsorbate_species, replace_vacancy=True,
                                                   adsorption_sites=sas,
                                                   heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       CatalystAdsorbateCrossover(),
])
op_selector = OperationSelector(*soclist)

For adsorabte-alloy surface systems, it is recommended to use a graph comparator to ensure there is no duplicates entering the population:

In [ ]:
comp = WLGraphComparator(dx=0.75)

Define the population:

In [ ]:
pop = Population(data_connection=db, population_size=pop_size, comparator=comp)
cc = GenerationRepetitionConvergence(pop, 2)

Define the energy minimization function, and calculate the fitness:

In [ ]:
def relax(atoms):
    # Centering the atoms
    atoms.center(vacuum=5., axis=2)
    # Here we use EMT calcualtor for efficiency
    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.1, steps=200)
    if not opt.converged():
      # Penalize unconverged structure
      e = 10000
    else:
      # Get the minimized EMT energy
      e = atoms.get_potential_energy()
    n_O = len([a for a in atoms if a.symbol=='O'])
    n_H = len([a for a in atoms if a.symbol=='H'])
    atoms.info['key_value_pairs']['raw_score'] = -(e - n_O*1.647 - (n_H-2*n_O)*(1.184-U))
    atoms.info['key_value_pairs']['EMT_energy'] = e
    # Save the adjacency matrix to speed up structure comparison
    nblist = neighbor_shell_list(atoms, dx=0.75, neighbor_number=1, mic=True)
    A = get_adj_matrix(nblist)
    atoms.info['data']['adj_matrix'] = A

Run the GA program:

In [ ]:
ncand = 1
f_gen1 = -1e5
atoms_gen1 = None
for atoms in db.get_all_unrelaxed_candidates():
    print('\rRelaxing candidate {}'.format(ncand), end='')
    relax(atoms)
    db.add_relaxed_step(atoms)
    f = atoms.info['key_value_pairs']['raw_score']
    if f > f_gen1:
      f_gen1 = f
      atoms_gen1 = atoms
    ncand += 1
pop.update()

f_gm = [f_gen1]
atoms_gm = [atoms_gen1]
max_gens = 200

# Below is the iterative part of the algorithm
gen_num = db.get_generation_number()
ncand = gen_num * pop_size + 1
for i in range(max_gens):
    # Check if converged
    if cc.converged():
        print('Converged')
        break

    print('\nCreating and evaluating generation {0}'.format(gen_num + i + 1))
    new_generation = []
    f_gen = -1e5
    atoms_gen = None
    for _ in range(pop_size):
        # Select an operator and use it
        op = op_selector.get_operator()
        # Select parents for a new candidate
        p1, p2 = pop.get_two_candidates()
        parents = [p1, p2]
        offspring, desc = op.get_new_individual(parents)
        # An operator could return None if an offspring cannot be formed
        # by the chosen parents
        if offspring is None:
            continue

        print('\rRelaxing candidate {0}'.format(ncand), end='')
        relax(offspring)
        f = offspring.info['key_value_pairs']['raw_score']
        # Update the global minimum of the current generation
        if f > f_gen:
            f_gen = f
            atoms_gen = offspring
        new_generation.append(offspring)
        ncand += 1

    # Append to the list of global optima
    f_gm.append(f_gen)
    atoms_gm.append(atoms_gen)
    # Plot the current optima
    x = list(range(gen_num + i + 2 - len(f_gm), gen_num + i + 2))
    y = -np.array(f_gm)
    plt.xticks(range(min(x), math.ceil(max(x)) + 1))
    plt.xlabel('Generation')
    plt.ylabel(r'Stability ($-f$)')
    plt.plot(x, y, marker='.', markersize=10)
    plt.show()

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    db.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update(new_generation)

Visualize the evolution of the global minimum structure:

In [ ]:
atoms_gm_single = atoms_gm.copy()
view(atoms_gm_single[-1], viewer='x3d')

## Global optimization under a range of reaction conditions

ACAT also implements an evolutionary multitasking (EM) algorithm, which can simultaneously progress multiple global optimization tasks. This is especially useful when one wants to derive phase diagrams that requires finding the most stable structures under a range of reaction conditions. In particular, the reaction condition should now be provided as a list or numpy array, and the population should now be an `acat.ga.MultitaskPopulation` instance.

As an example, we use the same setting as the previous section, but instead consider 41 different $U_{\mathrm{RHE}}$ conditions, ranging from -1 V to 1 V (0.05 V apart). Below is the adapted code for deriving the surface Pourbaix diagram of the PdAg$_3$(111) surface:

In [ ]:
# Define population
pop_size = 50
# Define the bimetallic species
bimetallic_species = ['Pd', 'Ag']
# Define the adsorbate species
adsorbate_species = ['O', 'H']
##################################################################################
# Define the reaction condition
U = np.arange(-1,1,0.05)
##################################################################################

# Initialize a pure fcc111 surface slab
proto = fcc111('Ag', (4,4,4), vacuum=5.)
# Change 1/4 Ag atoms to Pd
for i in [5,7,13,15,16,18,24,26,36,38,44,46,53,55,61,63]:
  proto[i].symbol = 'Pd'
proto_bot, proto_top = proto[:16], proto[16:]

# Generate 50 PdAg3(111) slabs with random orderings
rog = ROG(proto_top, elements=bimetallic_species,
      composition={'Pd':0.25,'Ag':0.75},
      trajectory='starting_slabs_multi.traj')
rog.run(num_gen=pop_size)

top_slabs = read('starting_slabs_multi.traj', index=':')
slabs = [proto_bot+a for a in top_slabs]
view(slabs)

# Get the adsorption sites. Composition does not affect GA operations
sas = SlabAdsorptionSites(slabs[0], surface='fcc111', ignore_sites='bridge',
              composition_effect=False)

# Get the atom indexes at different layers
layers = get_layers(slabs[0], (0, 0, 1), tolerance=0.3)[0] # Start from bottom
bot_indices = np.argwhere(layers == 0).ravel().tolist()
mut_indices = np.argwhere(layers > 0).ravel().tolist()

images = []
for slab in slabs:
  # Generate a random O* coverage pattern (set a random
  # min_adsorbate_distance to generate different coverages)
  image = mindcp(slab, adsorbate_species=adsorbate_species, adsorption_sites=sas,
           min_adsorbate_distance=np.random.uniform(2.2,5.2))
  images.append(image)
write('starting_generation_multi.traj', images)
view(images)

# Instantiate the db
db_name = 'PdAg3_multi.db'

os.system('rm {}'.format(db_name))
db = PrepareDB(db_name, cell=images[0].cell, population_size=pop_size)

for atoms in images:
    db.add_unrelaxed_candidate(atoms)

# Connect to the db
db = DataConnection(db_name)

soclist = ([3, 1, 1, 1, 1, 3],
       [RandomSlabPermutation(allowed_indices=mut_indices),
       AddAdsorbate(adsorbate_species, adsorption_sites=sas,
              max_coverage=1., min_adsorbate_distance=2.2,
              heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       RemoveAdsorbate(adsorbate_species, adsorption_sites=sas),
       MoveAdsorbate(adsorbate_species, adsorption_sites=sas,
              min_adsorbate_distance=2.2,
              heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       ReplaceAdsorbateSpecies(adsorbate_species, replace_vacancy=True,
                   adsorption_sites=sas,
                   heights={'ontop':1.85,'fcc':1.6,'hcp':1.6}),
       CatalystAdsorbateCrossover(),
])
op_selector = OperationSelector(*soclist)

comp = WLGraphComparator(dx=0.75)

##################################################################################
# Initialize the population and specify the number of tasks
pop = MultitaskPopulation(data_connection=db,
              population_size=pop_size,
              num_tasks=len(U),
              comparator=comp,
              exp_function=True,
              logfile='log.txt')
##################################################################################
cc = GenerationRepetitionConvergence(pop, 2)

def relax(atoms):
    # Centering the atoms
    atoms.center(vacuum=5., axis=2)
    # Here we use EMT calcualtor for efficiency
    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.1, steps=200)
    if not opt.converged():
      # Penalize unconverged structure
      e = 10000
    else:
      # Get the minimized EMT energy
      e = atoms.get_potential_energy()
    n_O = len([a for a in atoms if a.symbol=='O'])
    n_H = len([a for a in atoms if a.symbol=='H'])
    atoms.info['data']['raw_scores'] = -(e - n_O*1.647 - (n_H-2*n_O)*(1.184-U))
    atoms.info['key_value_pairs']['EMT_energy'] = e
    # Save the adjacency matrix to speed up structure comparison
    nblist = neighbor_shell_list(atoms, dx=0.75, neighbor_number=1, mic=True)
    A = get_adj_matrix(nblist)
    atoms.info['data']['adj_matrix'] = A

max_scores = np.repeat(-1e5, len(U))
max_indices = np.zeros(len(U), dtype=int)
all_images = []
first_generation = []
ncand = 1
for atoms in db.get_all_unrelaxed_candidates():
    print('\rRelaxing candidate {}'.format(ncand), end='')
    relax(atoms)
    scores = atoms.info['data']['raw_scores']
    max_indices[np.argwhere(scores>max_scores)] = 0*pop_size + ncand-1
    max_scores = np.maximum(scores, max_scores)
    all_images.append(atoms)
    first_generation.append(atoms)
    ncand += 1
images_gen = [all_images[idx] for idx in set(max_indices)]
# Update the population to allow new candidates to enter
# (DO NOT add relaxed candidates into db before this update)
pop.update(new_cand=first_generation)

max_gens = 200

# Below is the iterative part of the algorithm
gen_num = db.get_generation_number()
ncand = gen_num * pop_size + 1
for i in range(max_gens):
    # Check if converged
    if cc.converged():
        print('Converged')
        break

    print('\nCreating and evaluating generation {0}'.format(gen_num + i + 1))
    new_generation = []
    for j in range(pop_size):
        # Select an operator and use it
        op = op_selector.get_operator()
        # Select parents for a new candidate
        p1, p2 = pop.get_two_candidates()
        parents = [p1, p2]
        offspring, desc = op.get_new_individual(parents)
        # An operator could return None if an offspring cannot be formed
        # by the chosen parents
        if offspring is None:
            continue

        print('\rRelaxing candidate {0}'.format(ncand), end='')
        relax(offspring)
        scores = offspring.info['data']['raw_scores']
        max_indices[np.argwhere(scores>max_scores)] = (i+1)*pop_size + j
        max_scores = np.maximum(scores, max_scores)
        all_images.append(offspring)
        new_generation.append(offspring)
        ncand += 1

    # Plot the current surface phase diagram
    images_gen = [all_images[idx] for idx in set(max_indices)]
    for atoms in images_gen:
      scores = atoms.info['data']['raw_scores']
      plt.plot(U, -scores, label=atoms.get_chemical_formula(mode='hill'))
    plt.xlim(np.min(U), np.max(U))
    plt.xlabel(r'$U_{\rm{SHE}}$ / V')
    plt.ylabel(r'Stability ($-f$)')
    plt.title(r'PdAg$_3$(111) surface Pourbaix diagram (pH = 0) at generation {}'.format(gen_num+i))
    plt.legend(loc='upper right')
    plt.show()

    # Update the population to allow new candidates to enter
    # (DO NOT add relaxed candidates into db before this update)
    pop.update(new_cand=new_generation)

Finally, we can use the following script to get the most stable surface structure at each reaction condition and visualize them:

In [ ]:
from ase.db import connect
from ase.io import write
from ase.visualize import view
from ase.units import kB
import numpy as np

db = connect('PdAg3_multi.db')
fittest_images = []
seen_dns = set()
for row in db.select('relaxed=1'):
    atoms = row.toatoms()
    f_eff = row.raw_score
    dn = row.dominating_niche
    niches = row.data['niches']
    # Get the fittest individual with an effective
    # fitness of 0 in each niche (without duplicates)
    if (f_eff == 0) and (dn not in seen_dns):
        seen_dns.add(dn)
        atoms.info['data'] = {}
        atoms.info['data']['raw_scores'] = row.data['raw_scores']
        atoms.info['data']['niches'] = niches
        fittest_images.append(atoms)

fittest_images.sort(key=lambda x: x.info['data']['niches'][0])
for atoms in fittest_images:
    niches = atoms.info['data']['niches']
    scores = atoms.info['data']['raw_scores']
    label = atoms.get_chemical_formula(mode='hill')
    n_H = len([a for a in atoms if a.symbol == 'H'])
    n_O = len([a for a in atoms if a.symbol == 'O'])
    print(label + r' is the most stable structure at $U_{\rm{RHE}}$ = '
       + str(U[niches]) + ' with fitness of ' + str(scores[niches]))
view(fittest_images[-1], viewer='x3d')